In [0]:
DEBUG = False

OUTPUT_PATH = '/datascope/subaru/data/targeting/dSph/bootesi/netflow/bootes_1_003'

In [0]:
import os, sys
import re
from glob import glob
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

### Load config file

In [0]:
from pfs.ga.targeting.config import NetflowConfig
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.diagram import FOV, FP
from pfs.ga.targeting.instrument import *

In [0]:
fn = glob(os.path.join(OUTPUT_PATH, 'ga-netflow*.config'))[0]
config = NetflowConfig.from_file(fn, format='.yaml', ignore_collisions=True)
fn

In [0]:
config.instrument_options

# Validate web upload files

In [0]:
fn = glob(os.path.join(OUTPUT_PATH, f'{config.field.key}_assignments_web_*.feather'))[0]
df = pd.read_feather(fn)
df.columns

In [0]:
pd.options.display.max_columns = None
df.head(5)

In [0]:
df.tail(5)

# Validate PFS Design Files

### Load design files

In [0]:
from pfs.datamodel import PfsDesign

In [0]:
def get_pfsDesignId(fn):
    return int(re.search(r'pfsDesign-0x([0-9a-fA-F]{16})\.fits', fn).group(1), 16)

In [0]:
# Find design files
design_ids = []
for design_file in glob(os.path.join(OUTPUT_PATH, 'pfsDesign-*.fits')):
    design_ids.append(get_pfsDesignId(design_file))
    print(design_file, hex(design_ids[-1]))

In [0]:
pfsDesignId = design_ids[0]
pfsDesign = PfsDesign.read(pfsDesignId, dirName=OUTPUT_PATH)

In [0]:
# Verify that the instrument rotator is within limits
from pfs.utils.coordinates import DistortionCoefficients as DCoeff

ra = pfsDesign.raBoresight
dec = pfsDesign.decBoresight
posang = pfsDesign.posAng
obs_time = datetime(2025, 1, 24, 3, 30, 0) + timedelta(hours=10)

az, el, inr = DCoeff.radec_to_subaru(ra, dec, posang, obs_time,
                                     epoch=2000.0, pmra=0.0, pmdec=0.0, par=1e-6)

if inr < -174 or inr > 174:
    raise ValueError(f'Instrument rotator angle {inr} is out of range')

print(f'ra={ra}, dec={dec}, posang={posang}, obs_time={obs_time}')
print(f'az={az}, el={el}, inr={inr}')

In [0]:
# Instantiate instrument classes and diagrams

pointing = Pointing(ra, dec, posang=posang, obs_time=obs_time)
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
pfi = SubaruPFI(instrument_options=config.instrument_options)
fov = FOV(projection=wcs)
fp = FP(wfc)

### Plot design file

In [0]:
f, axs = plt.subplots(1, 2, figsize=(12, 6))

fp.plot(axs[0], pfsDesign.pfiNominal[..., 0], pfsDesign.pfiNominal[..., 1],
        native_frame='pixel', ms=1)

fov.plot(axs[1], pfsDesign.ra, pfsDesign.dec,
         native_frame='world', ms=1)

# Verify endpoint collisions

# Verify trajectory collisions

In [0]:
from ics.cobraOps.CollisionSimulator import CollisionSimulator
from ics.cobraOps.CollisionSimulator2 import CollisionSimulator2
from ics.cobraOps.TargetGroup import TargetGroup
from ics.cobraOps.cobraConstants import NULL_TARGET_POSITION, NULL_TARGET_ID
from pfs.utils.fiberids import FiberIds
from pfs.datamodel import FiberStatus, TargetType

In [0]:
pfi.bench, pfi.cobra_coach

In [0]:
# Positions are indexed by cobra ID but the design contains fiber IDs
# Cobra IDs are 1-based and go from 1..2394
# Some of the fibers are not connected to cobras

# Convert fiberIds to cobraIds
cobraids = pfi.fiber_map.fiberIdToCobraId(pfsDesign.fiberId)
cobra_mask = (cobraids != FiberIds.MISSING_VALUE)

# Look up line in design file by fiber ID
# Both maps are 1-based!
fiber_map = { pfsDesign.fiberId[i]: i for i in range(len(pfsDesign.fiberId)) }
cobra_map = { cobraids[i]: i for i in range(len(pfsDesign.fiberId)) if cobraids[i] != FiberIds.MISSING_VALUE}

print(cobraids, cobraids.min(), cobraids.max())
print(pfsDesign.fiberId.shape, cobra_mask.sum())

In [0]:
np.unique(pfsDesign.fiberStatus)

In [0]:
positions = np.full(pfi.bench.cobras.nCobras, NULL_TARGET_POSITION)
positions[cobraids[cobra_mask] - 1] = pfsDesign.pfiNominal[cobra_mask][..., 0] + \
                                      pfsDesign.pfiNominal[cobra_mask][..., 1] * 1j

# Remove unassigned fibers, broken fibers or broken cobras
mask = pfsDesign.targetType == TargetType.UNASSIGNED
positions[cobraids[cobra_mask & mask] - 1] = NULL_TARGET_POSITION

mask = pfsDesign.fiberStatus != FiberStatus.GOOD
positions[cobraids[cobra_mask & mask] - 1] = NULL_TARGET_POSITION

# Some cobras have no target and the focal plane positions given in the design file
# are the home position. Detect these because these might cause issues in trajectory
# calculations, and set them to the actual 64-bit home position
dist = np.abs(positions - pfi.bench.cobras.home0)
positions[dist < 1e-5] = pfi.bench.cobras.home0[dist < 1e-5]

# Some cobras have problems, for these, set the positions to the home position
# Do not set these, because they're already set to NULL_TARGET_POSITION
# These fibers are treated as bad by netflow and will be unassigned
# positions[pfi.bench.cobras.hasProblem] = pfi.bench.cobras.home0[pfi.bench.cobras.hasProblem]

print((dist < 1e-5).sum(), pfi.bench.cobras.hasProblem.sum())
print((positions == NULL_TARGET_POSITION).sum(), (pfsDesign.targetType[cobra_mask] == TargetType.UNASSIGNED).sum())

In [0]:
# Bad fibers
# BROKENFIBER = 2, "broken; ignore any flux"
# BLOCKED = 3, "temporarily blocked; ignore any flux"
# BLACKSPOT = 4, "hidden behind spot; ignore any flux"
# UNILLUMINATED = 5, "not illuminated; ignore any flux"
# BROKENCOBRA = 6, "Cobra does not move, but the fiber still carries flux."
# NOTCONVERGED = 7, "Cobra did not converge to the target."

mask = pfsDesign.fiberStatus != FiberStatus.GOOD
cobraids[mask], pfsDesign.fiberStatus[mask]

In [0]:
np.where(positions == NULL_TARGET_POSITION)

In [0]:
positions[46], positions[2223]

In [0]:
pfsDesign.pfiNominal[cobra_map[47]], pfsDesign.pfiNominal[cobra_map[2224]]

In [0]:
pfsDesign.pfiNominal[cobra_map[47]], pfsDesign.pfiNominal[cobra_map[2224]]

In [0]:
pfsDesign.objId[cobra_map[47]], pfsDesign.objId[cobra_map[2224]]

In [0]:
pfsDesign.ra[cobra_map[47]], pfsDesign.dec[cobra_map[47]], pfsDesign.ra[cobra_map[2224]], pfsDesign.dec[cobra_map[2224]]

In [0]:
pfsDesign.fiberHole[cobra_map[47]], pfsDesign.fiberHole[cobra_map[2224]]

In [0]:
pfsDesign.proposalId[cobra_map[47]], pfsDesign.proposalId[cobra_map[2224]]

In [0]:
pfsDesign.targetType[cobra_map[47]], pfsDesign.targetType[cobra_map[2224]]

In [0]:
# Verify that the positions are within the cobra range and cobras didn't get mis-indexed
np.abs((positions - pfi.bench.cobras.centers)[positions != NULL_TARGET_POSITION]).max()

In [0]:
# List index of unassigned or otherwise bad fibers
np.where(positions == NULL_TARGET_POSITION)[0]

In [0]:
# Verify that the final fiber positions are valid

theta, phi = pfi.bench.cobras.calculateRotationAngles(positions)
bad_positions = np.where((~np.isfinite(theta) | ~np.isfinite(phi)) & (positions != NULL_TARGET_POSITION))[0]

bad_positions

In [0]:
np.abs(positions[bad_positions] - pfi.bench.cobras.centers[bad_positions])

In [0]:
pfi.bench.cobras.L1, pfi.bench.cobras.L2

In [0]:
f, ax = plt.subplots(1, 1, figsize=(12, 12), dpi=240)

# Plot cobra IDs
for i in range(pfi.bench.cobras.nCobras):
    ax.text(pfi.bench.cobras.centers.real[i], pfi.bench.cobras.centers.imag[i], str(i + 1), fontsize=4, ha='center', va='center')

# Plot cobra centers
ax.plot(pfi.bench.cobras.centers.real, pfi.bench.cobras.centers.imag, '.k', ms=0.5, label='cobra  centers')

# Plot unassigned fibers
ax.plot(pfi.bench.cobras.centers.real[positions == NULL_TARGET_POSITION],
        pfi.bench.cobras.centers.imag[positions == NULL_TARGET_POSITION], 'ob', ms=3, label='unassigned')

# Plot broken cobras
ax.plot(pfi.bench.cobras.centers.real[pfi.bench.cobras.hasProblem],
        pfi.bench.cobras.centers.imag[pfi.bench.cobras.hasProblem], 'xr', ms=3, label='broken')

# Plot cobra home positions
ax.plot(pfi.bench.cobras.home0.real, pfi.bench.cobras.home0.imag, '.b', ms=.5, label='cobra home position')

ax.legend()

In [0]:
# Run with version 1

targets = TargetGroup(positions, ids=None, priorities=None)


simulator = CollisionSimulator(pfi.bench, targets, trajectorySteps=1000)

simulator.run()
if np.any(simulator.endPointCollisions):
    print("ERROR: detected end point collision, which should be impossible")
    print(np.where(simulator.endPointCollisions))

In [0]:
# List of broken cobras
np.where(pfi.bench.cobras.hasProblem)

In [0]:
# Run with version2

targets = TargetGroup(positions, ids=None, priorities=None)
simulator = CollisionSimulator2(pfi.bench, pfi.cobra_coach, targets)
simulator.run()

if np.any(simulator.endPointCollisions):
    print("ERROR: detected end point collision, which should be impossible")